In [1]:
import pandas as pd

# Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [10]:
rating = pd.read_csv('small_ratings.csv', sep=',')
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [19]:
df = rating.groupby('userId').agg({'timestamp':['min','max','count']}).reset_index()
df.head()

userId   timestamp                  
                 min         max count
0      1  1260759108  1260759205    20
1      2   835355395   835356246    76
2      3  1298861589  1298932787    51
3      4   949778714   949982274   204
4      5  1163373044  1163375145   100

In [61]:
# пользователей, которые выставили более 100 оценок
count100 = df.loc[lambda row: row.timestamp['count'] > 100 ]
count100.head()

userId   timestamp                  
                  min         max count
3       4   949778714   949982274   204
7       8  1154389340  1154474527   116
14     15   997937239  1469330735  1700
16     17  1127468587  1127476640   363
18     19   855190091   855195373   423

In [62]:
# среднее время жизни пользователей
(count100['timestamp']['max']-count100['timestamp']['min']).mean()

40080507.4496124

# Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

    rzd - железнодорожные перевозки
    auto - автомобильные перевозки
    air - воздушные перевозки
    client_base - адреса клиентов


In [50]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
#rzd

In [51]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
#auto

In [52]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
#air

In [53]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
#client_base

In [54]:
clients_types = (rzd
 .merge(auto, on='client_id',how='outer')
 .merge(air, on='client_id',how='outer')
 .fillna(0)
)

clients_types

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [49]:
clients_types.merge(client_base, on='client_id',how='inner')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


# Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

* Нужно ввести отдельный ID для уникального визита, чтобы учитывать логи одного и того же пользователя из разных мест

In [71]:
data = pd.DataFrame(
        {   'user_id': [1, 2, 1, 5, 67],
            'visit_id': [100, 101, 102, 103, 104],
            'coordinates': ['47.6647484 23.47438543', '34.353546 54.56478', '45.3425 38.65756', '44.6457658 77.35632', '65.45646 45.346346']
        }
)

data      

,user_id,visit_id,coordinates
0,1,100,47.6647484 23.47438543
1,2,101,34.353546 54.56478
2,1,102,45.3425 38.65756
3,5,103,44.6457658 77.35632
4,67,104,65.45646 45.346346


* Сами координаты можно отделньо проаналихзировать, например соединить с географической картой и определить регионы, города и т.д., так можно определить в каком городе что ищут, где больше всего польщователей и т.д. 